In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import Sequential, Model
from keras.layers import concatenate, Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, \
    GlobalAveragePooling2D, AveragePooling2D, Input, Add, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.utils import np_utils
from keras.models import load_model
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import cv2
import os
import pickle

2022-11-26 19:03:29.437784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 19:03:29.437810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def load_wav(path):
    return librosa.core.load(path, sr=24000)[0]

def cough_s(wav_path):
    sr = 24000
    wav = load_wav(wav_path)
    cough_segments, cough_mask = segment_cough(wav, sr, cough_padding=0)
    return cough_segments, cough_mask

In [4]:
def segment_cough(x, fs, cough_padding=0.2, min_cough_len=0.2, th_l_multiplier=0.1, th_h_multiplier=2):
    cough_mask = np.array([False] * len(x))

    # Define hysteresis thresholds
    rms = np.sqrt(np.mean(np.square(x)))
    seg_th_l = th_l_multiplier * rms
    seg_th_h = th_h_multiplier * rms
    # 기침소리 크기 컷팅을 위한 최저,최고 임계값

    # Segment coughs-> 기침소리 + 작은기침소리(+잡음)에서 앞부분만 자르기 위함
    coughSegments = []
    padding = round(fs * cough_padding)
    # 기침 컷팅 후 전후의 추가되는 샘플수
    min_cough_samples = round(fs * min_cough_len)
    # 기침의 최소샘플수
    cough_start = 0
    cough_end = 0
    cough_in_progress = False
    tolerance = round(0.01 * fs)
    below_th_counter = 0
    # 기침 진행 중 sample 값이 low임계값보다 작은 횟수

    for i, sample in enumerate(x ** 2):
        # enumerate를 사용해 현재 반복 횟수(i)와 현재 반복의 항목 값(sample) 저장
        if cough_in_progress:
            if sample < seg_th_l:
                below_th_counter += 1
                if below_th_counter > tolerance:
                    cough_end = i + padding if (i + padding < len(x)) else len(x) - 1
                    cough_in_progress = False
                    if (cough_end + 1 - cough_start - 2 * padding > min_cough_samples):
                        coughSegments.append(x[cough_start:cough_end + 1])
                        cough_mask[cough_start:cough_end + 1] = True
            elif i == (len(x) - 1):
                cough_end = i
                cough_in_progress = False
                if (cough_end + 1 - cough_start - 2 * padding > min_cough_samples):
                    coughSegments.append(x[cough_start:cough_end + 1])
            else:
                below_th_counter = 0
        else:
            if sample > seg_th_h:
                cough_start = i - padding if (i - padding >= 0) else 0
                cough_in_progress = True

            # think- 앞에 패딩을 해주기위한 과정인 거 같은데, 만약 기침 시작을 녹음과 동시에 시작으로 해서 기침 시작 sample의 index가 padding의 못 미치면?

    return coughSegments, cough_mask

def compute_SNR(x, fs):
    """Compute the Signal-to-Noise ratio of the audio signal x (np.array) with sampling frequency fs (float)"""
    segments, cough_mask = segment_cough(x, fs)
    RMS_signal = 0 if len(x[cough_mask]) == 0 else np.sqrt(np.mean(np.square(x[cough_mask])))
    RMS_noise = np.sqrt(np.mean(np.square(x[~cough_mask])))
    SNR = 0 if (RMS_signal == 0 or np.isnan(RMS_noise)) else 20 * np.log10(RMS_signal / RMS_noise)
    return SNR

In [5]:
# librosa를 사용한 7가지 오디오 특징 추출 및 특징 set 만들기
def feature_extractor_rec(row):
    recpath = '/home/pmp/Workspace/Sera/COVID-19/model_test/'
    sr = 24000

    name = row[0]
    s=[]
    c=[]
    try:
        # audio,sr = librosa.load(row[-2])
        # For MFCCS
        audio = row[1]

        #1.mfcc
        mfccs    = librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=13)
        mfccsscaled = np.mean(mfccs.T,axis=0)

        #2.delta
        delta    = librosa.feature.delta(mfccs)
        deltascaled = np.mean(delta.T,axis=0)

        # mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        # mfccsscaled = np.mean(mfccs.T, axis=0)
        # s.append(0)
        # # 열을 따라 산술 평균을 계산

        # cent = librosa.feature.spectral_centroid(y=audio, sr=sr)
        # centscaled = np.mean(cent.T, axis=0)
        # s.append(1)

        # bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
        # bandwidthscaled = np.mean(bandwidth.T, axis=0)
        # s.append(2)

        # contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        # contrastscaled = np.mean(contrast.T, axis=0)
        # s.append(3)

        # flatness = librosa.feature.spectral_flatness(y=audio)
        # flatnessscaled = np.mean(flatness.T, axis=0)
        # s.append(4)

        # rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
        # rolloffscaled = np.mean(rolloff.T, axis=0)
        # s.append(5)

        # chroma_vec = librosa.feature.chroma_stft(y=audio, sr=sr)
        # chroma_vecscaled = np.mean(chroma_vec.T, axis=0)  # 12
        # s.append(6)

        # Feature_Set_rec = np.concatenate(
        #     (mfccsscaled, centscaled, bandwidthscaled, contrastscaled, flatnessscaled, rolloffscaled, chroma_vecscaled),
        #     axis=None)

        Feature_Set_rec =  np.concatenate([mfccsscaled, deltascaled],axis=None)


        # Spectogram
        plt.axis('off')  # no axis
        c.append(0)

        plt.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        c.append(1)
        melspec = librosa.feature.melspectrogram(y=audio, sr=sr)
        c.append(2)
        s_db = librosa.power_to_db(melspec, ref=np.max)
        c.append(3)
        librosa.display.specshow(s_db)
        c.append(4)
        savepath_rec = os.path.join(recpath, name + '.png')
        c.append(5)
        plt.savefig(savepath_rec)
        # 여백 있게 그래프 저장
        c.append(6)
        plt.close()

    except:
        print('File cannot open:', name)
        print('s: ',s)
        print('c: ',c)

        return None, None
    return Feature_Set_rec, savepath_rec

In [12]:
def mod(cough_segments):
    MERGM_f = load_model('/home/pmp/Workspace/Sera/COVID-19/model/2000_record_mfcc+delta_epoch40.h5')

    DATA = {}
    DATA['MFCCS'] = {}
    DATA['MEL'] = {}
    DATA['LABELS'] = {}

    # with open('C:/Workspace/covid/loaded_data_model2_seg_1041_2480_mfcc13.pickle', 'rb') as af:
    #     # 파일을 열고 닫는 것을 자동으로 처리
    #     DATA = pickle.load(af)

    # 받은 데이터 잘라내기
    data_seg_rec = {}
    data_seg_rec['id'] = []
    data_seg_rec['data'] = []
    data_seg_rec['covid_status'] = []

    for j in range(len(cough_segments)):
        data_seg_rec['id'].append(str(j))
        data_seg_rec['covid_status'].append('Unknown')
        data_seg_rec['data'].append(cough_segments[j])

    ds_rec = pd.DataFrame.from_dict(data_seg_rec)
    # data segement를 받아 DataFrame반환 = ds_rec

    features_rec = []
    imgpaths_rec = []

    for row in ds_rec.values:
        feature_set_rec, savepath_rec = feature_extractor_rec(row)

        if any(feature_set_rec) is None:
            return None

        features_rec.append(feature_set_rec)
        imgpaths_rec.append(savepath_rec)

    isnone = lambda x: x is not None
    label = lambda x: 3 if x == 'Unknown' else 0
    cast_x = list(map(isnone, features_rec))
    data_y = list(map(label, ds_rec['covid_status']))

    data_x = [features_rec[i] for i in range(len(features_rec)) if cast_x[i] == True]
    data_xx = [imgpaths_rec[i] for i in range(len(imgpaths_rec)) if cast_x[i] == True]
    data_y = [data_y[i] for i in range(len(features_rec)) if cast_x[i] == True]

    indices = np.arange(len(data_x))

    # 녹음된 data에 대한 처리
    DATA_rec = {}
    DATA_rec['MFCCS'] = {}
    DATA_rec['MEL'] = {}
    DATA_rec['LABELS'] = {}

    DATA_rec['MFCCS'] = np.array([data_x[i] for i in indices])
    DATA_rec['MEL'] = [data_xx[i] for i in indices]
    DATA_rec['LABELS'] = np.array([data_y[i] for i in indices])

    ###
    test_features = DATA_rec['MFCCS']
    test_imgs = DATA_rec['MEL']
    test_labels = DATA_rec['LABELS']

    TEST = TripleInputGenerator(test_features, test_imgs, test_labels, batch_size=48, target_size=(224, 224))

    np.set_printoptions(precision=6, suppress=True)

    return MERGM_f.predict(TEST)

In [13]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, imgfiles, labels, batch_size, target_size=(224, 224), shuffle=False, scale=255, n_classes=1,
                 n_channels=3):
        self.batch_size = batch_size
        self.dim = target_size
        self.labels = labels
        self.imgfiles = imgfiles
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.n_channels = n_channels
        self.scale = scale

        self.c = 0
        self.on_epoch_end()

    def __len__(self):
        # returns the number of batches
        if len(self.imgfiles) <= 10:
            return int(len(self.imgfiles))
        else:
            int(np.floor(len(self.imgfiles) / self.batch_size))

    def __getitem__(self, index):
        # returns one batch
        if len(self.indexes) <= 10:
            indexes = self.indexes
        else:
            indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.imgfiles))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        if len(self.indexes) <= 10:
            X = np.empty((len(self.indexes), *self.dim, self.n_channels))
            y = np.empty(len(self.indexes), dtype=int)
        else:
            X = np.empty((self.batch_size, *self.dim, self.n_channels))
            y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store list_IDs_temp
            img = cv2.imread(self.imgfiles[ID])
            img = cv2.resize(img, self.dim, interpolation=cv2.INTER_CUBIC)
            X[i,] = img / self.scale

            # Store class
            y[i] = self.labels[ID]

            self.c += 1
        return X, y  # keras.utils.to_categorical(y, num_classes=self.n_classes)

class CustomPipeline(tf.keras.utils.Sequence):
    def __init__(self, data_x, data_y, batch_size=48, shuffle=False, n_classes=1):
        self.features = data_x
        self.labels = data_y
        self.batch_size = 48
        self.shuffle = shuffle
        self.n_features = self.features.shape[1]  # 36
        self.n_classes = 1
        self.on_epoch_end()

    def __len__(self):
        if len(self.features) <= 10:
            return int(len(self.features))
        else:
            return int(np.floor(len(self.features) / self.batch_size))

    def __getitem__(self, index):
        if len(self.indexes) <= 10:
            indexes = self.indexes
        else:
            indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.features))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        if len(self.indexes) <= 10:
            X = np.empty((len(self.indexes), self.n_features))
            y = np.empty(len(self.indexes), dtype=int)
        else:
            X = np.empty((self.batch_size, self.n_features))
            y = np.empty((self.batch_size), dtype=int)

        for i, ID in enumerate(indexes):
            X[i,] = self.features[ID]
            y[i,] = self.labels[ID]
        return X, y

In [14]:
class TripleInputGenerator(tf.keras.utils.Sequence):
    # Wrapper of two generatos for the combined input model
    def __init__(self, X1, X2, Y, batch_size, target_size=(224, 224)):
        self.genX1 = CustomPipeline(X1, Y, batch_size=batch_size, shuffle=False)
        self.genX2 = CustomDataset(X2, Y, batch_size=batch_size, shuffle=False, target_size=target_size)

    def __len__(self):
        return self.genX1.__len__()

    def __getitem__(self, index):
        X1_batch, Y_batch = self.genX1.__getitem__(index)
        X2_batch, Y_batch = self.genX2.__getitem__(index)

        X_batch = [X1_batch, X2_batch]
        return X_batch, Y_batch

*** Testing 1 data ***

In [24]:
# # wav = 'C:/Workspace/covid/cough/user_input_w/covid/c000_015576c8-2b03-4dcb-b251-78a984fe86fe.wav'
# # wav = '/home/pmp/Workspace/Sera/COVID-19/dataset/originaldata_440/covid/c000_015576c8-2b03-4dcb-b251-78a984fe86fe.wav'
# wav='/home/pmp/Workspace/Sera/dataset/record_normalized_dataset/coswara2_covid (2).wav'
# cough_segments, cough_mask = cough_s(wav)

# y_preds_test = mod(cough_segments)
# y_preds_test

array([[1.]], dtype=float32)

In [25]:
# preds = [pred for pred in y_preds_test if pred > 0.9]

# print('preds : ', len(preds))
# print('y_preds_test : ', y_preds_test)
# print()
# if (len(y_preds_test) / 2) <= len(preds):
#     print("positive")
# else:
#     print("negative")

preds :  1
y_preds_test :  [[1.]]

positive


# 민감도

양성을 양성으로 판단할 확률

(covid -> covid)

In [20]:
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction=0.4
# session = tf.Session(config=config)

# wav_dir = '/home/pmp/Workspace/Sera/COVID-19/dataset/originaldata_440/covid/'
wav_dir = '/home/pmp/Workspace/Sera/nomalized2000/covid/'
file_list = os.listdir(wav_dir)

path = []
for j in file_list:
    path.append(wav_dir + str(j))
#print(path)

In [ ]:
positive_cnt_1 = 0
negative_cnt_1 = 0

for i in path:
    filenum = i[70:74]
            
    cough_segments, cough_mask = cough_s(i)
    try:
        y_preds_test = mod(cough_segments)

        preds = [pred for pred in y_preds_test if pred > 0.9]

        if (len(y_preds_test) / 2) <= len(preds):
#         if len(preds) > 0:
            positive_cnt_1 = positive_cnt_1+1
            print(filenum+"// positive_cnt = " , positive_cnt_1)

        else:
#             negative_cnt = negative_cnt+1
            negative_cnt_1 = negative_cnt_1+1
            print(filenum+"// negative_cnt = " , negative_cnt_1)
    
    except:
        print("pass : ", filenum)
        
# sensitivity = positive_cnt_1 / (positive_cnt_1 + negative_cnt_1)
# print("---------------------------------------------------")
# print("total (except pass) = ", positive_cnt_1+negative_cnt_1)
# print("correct data (positive) = ", positive_cnt_1)
# print("wrong data (negative) = ", negative_cnt_1)
# print("---------------------------------------------------")
# print("(민감도) SENSITIVITY = ", sensitivity)
# print("---------------------------------------------------")

pass :  
// positive_cnt =  1
// positive_cnt =  2
// positive_cnt =  3
// positive_cnt =  4
// positive_cnt =  5
// positive_cnt =  6
pass :  
// positive_cnt =  7
// positive_cnt =  8
// positive_cnt =  9
// positive_cnt =  10
// negative_cnt =  1
// positive_cnt =  11
// positive_cnt =  12
// positive_cnt =  13
// positive_cnt =  14
// positive_cnt =  15
// positive_cnt =  16
// positive_cnt =  17
pass :  
pass :  
// positive_cnt =  18
// positive_cnt =  19
// positive_cnt =  20
// positive_cnt =  21
// positive_cnt =  22
// positive_cnt =  23
// positive_cnt =  24


In [ ]:
sensitivity = positive_cnt_1 / (positive_cnt_1 + negative_cnt_1)
print("---------------------------------------------------")
print("total (except pass) = ", positive_cnt_1+negative_cnt_1)
print("correct data (positive) = ", positive_cnt_1)
print("wrong data (negative) = ", negative_cnt_1)
print("---------------------------------------------------")
print("(민감도) SENSITIVITY = ", sensitivity)
print("---------------------------------------------------")

## 특이도

음성을 음성으로 판단할 확률

(healthy -> healthy)

In [ ]:
wav_dir = '/home/pmp/Workspace/Sera/nomalized2000/healthy/'

file_list = os.listdir(wav_dir)

path = []
for j in file_list:
    path.append(wav_dir + str(j))
    
path.sort()
        
positive_cnt_2 = 0
negative_cnt_2 = 0

for i in path:

    cough_segments, cough_mask = cough_s(i)
    try:
        y_preds_test = mod(cough_segments)

        preds = [pred for pred in y_preds_test if pred > 0.9]

        if (len(y_preds_test) / 2) < len(preds):
#         if len(preds) > 0:
            positive_cnt_2 = positive_cnt_2+1
            print(i[70:85]+"// positive_cnt = " , positive_cnt_2)

        else:
            negative_cnt_2 = negative_cnt_2+1
            print(i[70:85]+"// negative_cnt = " , negative_cnt_2)
    
    except:
        print("pass : ", i[70:85])
        
specificity = negative_cnt_2 / (positive_cnt_2 + negative_cnt_2)
print("---------------------------------------------------")
print("total (except pass) = ", positive_cnt_2+negative_cnt_2)
print("correct data (negative) = ", negative_cnt_2)
print("wrong data (positive) = ", positive_cnt_2)
print("---------------------------------------------------")
print("(특이도) SPECIFICITY = ", specificity)
print("---------------------------------------------------")

2022-10-18 12:52:59.141182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 12:52:59.141605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pmp/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-18 12:52:59.141710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pmp/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-18 12:52:59.141807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so

// negative_cnt =  1
pass :  v
av// negative_cnt =  2
av// positive_cnt =  1
av// negative_cnt =  3
av// negative_cnt =  4
av// negative_cnt =  5
av// negative_cnt =  6
av// positive_cnt =  2
pass :  av
pass :  av
av// positive_cnt =  3
pass :  v
av// negative_cnt =  7
av// negative_cnt =  8
av// positive_cnt =  4
av// negative_cnt =  9
av// negative_cnt =  10
av// negative_cnt =  11
av// negative_cnt =  12
av// negative_cnt =  13
av// negative_cnt =  14
av// negative_cnt =  15
pass :  v
av// negative_cnt =  16
av// negative_cnt =  17
av// negative_cnt =  18
pass :  av
av// positive_cnt =  5
av// negative_cnt =  19
pass :  av
av// negative_cnt =  20
av// negative_cnt =  21
av// negative_cnt =  22
v// negative_cnt =  23
av// negative_cnt =  24
av// negative_cnt =  25
av// negative_cnt =  26
av// negative_cnt =  27
av// negative_cnt =  28
av// negative_cnt =  29
av// negative_cnt =  30
av// negative_cnt =  31
av// negative_cnt =  32
av// negative_cnt =  33
pass :  v
pass :  av
pass :  av

// negative_cnt =  224
// negative_cnt =  225
// negative_cnt =  226
// negative_cnt =  227
// negative_cnt =  228
// negative_cnt =  229
// negative_cnt =  230
// negative_cnt =  231
// negative_cnt =  232
// negative_cnt =  233
pass :  
// negative_cnt =  234
// negative_cnt =  235
// negative_cnt =  236
// negative_cnt =  237
// negative_cnt =  238
// negative_cnt =  239
// negative_cnt =  240
// negative_cnt =  241
// negative_cnt =  242
// negative_cnt =  243
// negative_cnt =  244
// negative_cnt =  245
// negative_cnt =  246
// negative_cnt =  247
// negative_cnt =  248
// negative_cnt =  249
// negative_cnt =  250
// negative_cnt =  251
// negative_cnt =  252
// negative_cnt =  253
// negative_cnt =  254
// negative_cnt =  255
// negative_cnt =  256
// negative_cnt =  257
// negative_cnt =  258
// negative_cnt =  259
// negative_cnt =  260
// negative_cnt =  261
// negative_cnt =  262
// negative_cnt =  263
// negative_cnt =  264
// negative_cnt =  265
// negative_cnt =  266
//

// negative_cnt =  571
// negative_cnt =  572
// negative_cnt =  573
// negative_cnt =  574
// negative_cnt =  575
// negative_cnt =  576
// negative_cnt =  577
// negative_cnt =  578
// negative_cnt =  579
// negative_cnt =  580
// negative_cnt =  581


## 정확도

In [ ]:
a = (positive_cnt_1 + negative_cnt_2) /(positive_cnt_1 + negative_cnt_1 + positive_cnt_2 + negative_cnt_2)

print(a)

출력창 테스트용

In [22]:
#import os

wav_dir = '/home/pmp/Workspace/Sera/COVID-19/dataset/originaldata_440/healthy/c000_015576c8-2b03-4dcb-b251-78a984fe86fe.wav'
print(wav_dir[67:72])

c000_


In [ ]:
import os

wav_dir = 'C:/Workspace/covid/cough/user_input_w/healthy/'
file_list = os.listdir(wav_dir)

path = []
for j in file_list:
    path.append(wav_dir + str(j))
    
#print(path)
        
positive_cnt = 0
negative_cnt = 0

for i in path:
    print(i[46:51]+"// positive_cnt = " , positive_cnt)

['C:/Workspace/covid/cough/user_input_w/healthy/h0038_0272a923-9d72-4566-abfa-9cfb23be2b4a.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0039_027887b8-1da6-477a-be4c-0f11958f1eac.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0040_02796c32-e748-4b86-b7be-06e184e120aa.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0041_027a2c6c-067e-426f-9b5c-c3352a27c549.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0042_02909ffd-16cb-47d6-9592-b30b1558227f.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0043_029f6ee9-d7c5-4d4c-a153-09c210683687.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0044_02b80927-ead6-492b-aca9-33a009050c27.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0045_02def005-fb2b-46aa-97ce-66262fb6f93d.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0046_02e59170-07cd-49eb-8d1f-59f9b0a0bec7.wav', 'C:/Workspace/covid/cough/user_input_w/healthy/h0047_02fc2085-a18c-4f46-a559-94f0e85d5496.wav', 'C:/Workspace/covid/cough/user_input_w/